In [28]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
from collections import defaultdict
import re
from tqdm import tqdm
import time
from random import randint, random
import undetected_chromedriver as uc
from selenium import webdriver
import selenium.webdriver as webdriver

browser = uc.Chrome(use_subprocess=True) ##use_subprocess=True para uc.Chrome hay que tener cuidado con las updates
website="https://www.amazon.co.uk"
browser.get(website)
browser.set_page_load_timeout(10)
browser.maximize_window()
wait = WebDriverWait(browser, 10)

##Logs into Account
browser.find_element(By.ID , "nav-link-accountList-nav-line-1").click()

username = browser.find_element(By.ID , "ap_email")

username.send_keys("ariel.hernan.limes@pwc.com") ##Here goes the email account
continue_button = browser.find_element(By.ID, "continue")
continue_button.click()

password = browser.find_element(By.ID , "ap_password")
password.send_keys("CompraLicencias") ##Here goes the password
login_button = browser.find_element(By.ID, "signInSubmit")
login_button.click()

browser.refresh()
time.sleep(random()*3)

location=wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="nav-line-2 nav-progressive-content"]')))
location.click()

address_input=wait.until(EC.presence_of_element_located((By.XPATH, '//input[@id="GLUXZipUpdateInput"]')))
address_input.clear()
address_input.send_keys("AB11 6DA")
address_input.send_keys(Keys.ENTER)
time.sleep(2)

brand_url='https://www.amazon.co.uk/stores/page/BE0B08DB-4583-4C8A-A689-CDE644FC7629?ingress=0&visitId=53b3f803-1726-4a1a-88ea-720b07c22064'
browser.get(brand_url)

In [29]:
data_dict={
    'product_title': [],
    'rating': [],    
    'n_rating': [],
    'original_price': [],
    'begin_price': [],
    'to_price': [],
}

def get_info(product):
    
    global data_dict

    # **************************************************************************************************************************************
    try:
        product_title=product.find_element(By.CSS_SELECTOR,"[data-testid='product-grid-title']").text
    except:
        product_title="No title"
    
    # ***************************************************************************************************************************************
    
    try:
        element = product.find_element(By.CSS_SELECTOR,'[data-testid="icon-star"]')
    
        stars_text = browser.execute_script('return arguments[0].textContent;', element)
        match = re.search(r'(\d+\.\d+)', stars_text)
        if match:
            rating = float(match.group(1))
        else:
            rating = 0
    except:
        rating=0
    
    # ***************************************************************************************************************************************
    
    try:
        n_rating=product.find_element(By.CSS_SELECTOR,"[data-testid='grid-item-review-count']").text
    except:
        n_rating=0
    
    # ***************************************************************************************************************************************
    
    try:
        element = product.find_element(By.CSS_SELECTOR,"[class*='Price__price__LKpWT Price__small__Y4NDm']")
        orig_prices = browser.execute_script('return arguments[0].textContent;', element)

        pattern = r'£([\d.]+)'
        matches = re.search(pattern, orig_prices)
        if matches:
            original_price = matches.group(1)
        else:
            original_price = 0
    except:
        original_price=0    
    
    # ***************************************************************************************************************************************
    try:
        element = product.find_element(By.CSS_SELECTOR,'[data-testid="grid-item-buy-price"]')
            
        prices = browser.execute_script('return arguments[0].textContent;', element)
        pattern = r'£([\d.]+)(?: – £([\d.]+))?|£([\d.]+)RRP:£([\d.]+)'

    # Match the pattern in the input string
        matches = re.search(pattern, prices)

        if matches:
            if matches.group(1):
                group1 = matches.group(1)
                group2 = matches.group(2) if matches.group(2) else '0'
                begin_price= group1
                to_price= group2
            elif matches.group(3):
                group1 = matches.group(3)
                group3 = matches.group(4)
                begin_price = group1   
        else:
            begin_price=0
            to_price=0
            
    except:
        begin_price=0
        to_price=0


    # ***************************************************************************************************************************************
    
    
    data_dict['product_title'].append(product_title)
    data_dict['rating'].append(rating)
    data_dict['n_rating'].append(n_rating)
    data_dict['begin_price'].append(begin_price)
    data_dict['to_price'].append(to_price)
    data_dict['original_price'].append(original_price)
    
    
def scrolling():
    
    browser.execute_script("document.body.style.overflow = 'scroll';")
    browser.execute_script("document.body.style.position = 'relative';")
    time.sleep(1) 
    # Scroll
    for scroll in range(0,50):
        body = browser.find_element(by=By.XPATH, value='/html/body')
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)
        
    time.sleep(random()*5)
    


def getting_data():
    product_wrapper = browser.find_elements(By.XPATH, '//li[@data-testid="product-grid-item"]')
    for product in product_wrapper:
        get_info(product)

In [30]:
scrolling()

getting_data()

In [32]:
df=pd.DataFrame(data_dict)
df

,product_title,rating,n_rating,original_price,begin_price,to_price
0,Under Armour Men's UA Storm Liner,4.5,578,0,17.17,47.00
1,Under Armour Men's M's Training,4.4,70,0,18.40,47.00
2,Under Armour Men's Blitzing Cap Adjustable,4.3,56,0,15.92,71.00
3,Under Armour Men's Blitzing Cap Adjustable,4.5,52,0,15.92,71.00
4,Under Armour Men's Blitzing Cap Adjustable,4.7,104,0,15.92,71.00
...,...,...,...,...,...,...
153,Under Armour Women's Ua Charged Pursuit 3 Meta...,4.4,837,0,28.00,149.00
154,Under Armour Women's Ua W Charged Pursuit 3 Bl...,3.5,2,0,43.53,129.76
155,Under Armour Women's Ua W Surge 3 Running Shoe,4.3,641,0,23.00,126.00
156,Under Armour Women's Surge 3 Running Shoe,4.5,392,0,36.95,146.88
